Multi-platform loading

In [1]:

###SETTINGS###

import os
import subprocess
import shutil
from getpass import getpass

# Platform detection
ON_KAGGLE = os.path.exists('/kaggle')
ON_COLAB = 'COLAB_RELEASE_TAG' in os.environ or os.path.exists('/content')

if shutil.which("aria2c") is None:
    print("aria2c not found → installing...")
    try:
        subprocess.run(["apt", "update", "-qq"], check=True, capture_output=True)
        result = subprocess.run(["apt", "install", "-y", "-qq", "aria2"], capture_output=True, text=True)
        if result.returncode == 0:
            print("aria2c installed successfully")
        else:
            print("Install failed (code {}):".format(result.returncode))
            print("stderr:", result.stderr.strip())
    except subprocess.CalledProcessError as e:
        print(f"apt error (code {e.returncode}): {e.stderr}")
    except Exception as e:
        print(f"Unexpected error: {e}")
else:
    print("aria2c already available")

# Determining the working directory
possible_bases = [
    "/workspace",       # Vast.ai / RunPod
    "/kaggle/working",  # Kaggle
    "/content",         # Google Colab
]

BASE_DIR = None
for path in possible_bases:
    if os.path.isdir(path):
        BASE_DIR = path
        break

if BASE_DIR is None:
    BASE_DIR = os.getcwd()
    print("WARNING: Known directory not found:", BASE_DIR)

print("Working directory:", BASE_DIR)

# Configuration
FORGE_DIR        = os.path.join(BASE_DIR, "stable-diffusion-webui-forge")
MODELS_DIR       = os.path.join(BASE_DIR, "stable-diffusion-webui-forge", "models", "Stable-diffusion")
LORA_DIR         = os.path.join(BASE_DIR, "stable-diffusion-webui-forge", "models", "Lora")
CONTROLNET_DIR   = os.path.join(FORGE_DIR, "extensions", "sd-webui-controlnet")
CONTROLNET_MODELS_DIR = os.path.join(CONTROLNET_DIR, "models")
EXTENSIONS_DIR   = os.path.join(FORGE_DIR, "extensions")
OUTPUTS_DIR      = os.path.join(FORGE_DIR, "outputs")

for d in [MODELS_DIR, LORA_DIR, CONTROLNET_DIR, CONTROLNET_MODELS_DIR, EXTENSIONS_DIR, OUTPUTS_DIR]:
    os.makedirs(d, exist_ok=True)


def get_secret(name: str):
    """Get secret from env/Kaggle/Colab, then optional prompt."""
    value = os.environ.get(name)
    if value:
        return value.strip()

    # Kaggle secrets
    if ON_KAGGLE:
        try:
            from kaggle_secrets import UserSecretsClient
            value = UserSecretsClient().get_secret(name)
            return value.strip() if value else None
        except Exception:
            pass

    # Colab secrets panel: from google.colab import userdata
    if ON_COLAB:
        try:
            from google.colab import userdata
            value = userdata.get(name)
            if value:
                return value.strip()
        except Exception:
            pass

    # Manual fallback for Vast.ai/local
    try:
        return getpass(f"{name}: ").strip() or None
    except Exception:
        return None


CIVITAI_TOKEN = get_secret("CIVITAI_TOKEN")
HF_TOKEN = get_secret("HF_TOKEN")

TOKENS = {}
if CIVITAI_TOKEN:
    TOKENS["CIVITAI"] = CIVITAI_TOKEN
if HF_TOKEN:
    TOKENS["HF"] = HF_TOKEN

if not CIVITAI_TOKEN:
    print("CivitAI token not found")
if not HF_TOKEN:
    print("HF token not found")



# Download function
from urllib.parse import urlencode


def _prepare_download_url(url, token):
    """CivitAI download works more reliably with token as query param."""
    if token and "civitai.com/api/download/models" in url and "token=" not in url:
        sep = "&" if "?" in url else "?"
        return f"{url}{sep}{urlencode({'token': token})}"
    return url


def _run_cmd(cmd):
    return subprocess.run(cmd, check=True, capture_output=True, text=True)


def _looks_valid_file(path):
    return os.path.exists(path) and os.path.getsize(path) > 1_000_000


def download_file(url, output_path, token=None):
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    if _looks_valid_file(output_path):
        return

    token = token.strip() if isinstance(token, str) else token
    final_url = _prepare_download_url(url, token)

    attempts = []

    # On Kaggle, CivitAI frequently fails with aria2c/wget but succeeds with curl.
    prefer_curl_first = ON_KAGGLE and "civitai.com/api/download/models" in final_url and shutil.which("curl")

    if prefer_curl_first:
        curl_cmd = [
            "curl", "-L", "-C", "-", "--fail", "--silent", "--show-error",
            "--retry", "3", "--retry-delay", "2",
            "-A", "Mozilla/5.0",
            final_url, "-o", output_path,
        ]
        if token:
            curl_cmd = curl_cmd[:1] + ["-H", f"Authorization: Bearer {token}"] + curl_cmd[1:]
        attempts.append(("curl", curl_cmd))

    if shutil.which("aria2c"):
        aria2_cmd = [
            "aria2c", "-x", "16", "-s", "16", "-k", "1M",
            "--summary-interval=0", "--console-log-level=error",
            "-d", os.path.dirname(output_path),
            "-o", os.path.basename(output_path),
            "--max-tries=2", "--retry-wait=2",
            final_url,
        ]
        if token:
            aria2_cmd += ["--header", f"Authorization: Bearer {token}"]
        attempts.append(("aria2c", aria2_cmd))

    wget_cmd = [
        "wget", "-c", "-q", final_url, "-O", output_path,
        "--tries=2", "--waitretry=2",
        "--user-agent=Mozilla/5.0",
    ]
    if token:
        wget_cmd = wget_cmd[:3] + ["--header", f"Authorization: Bearer {token}"] + wget_cmd[3:]
    attempts.append(("wget", wget_cmd))

    if not prefer_curl_first and shutil.which("curl"):
        curl_cmd = [
            "curl", "-L", "-C", "-", "--fail", "--silent", "--show-error",
            "--retry", "3", "--retry-delay", "2",
            "-A", "Mozilla/5.0",
            final_url, "-o", output_path,
        ]
        if token:
            curl_cmd = curl_cmd[:1] + ["-H", f"Authorization: Bearer {token}"] + curl_cmd[1:]
        attempts.append(("curl", curl_cmd))

    failures = []
    for tool_name, cmd in attempts:
        try:
            _run_cmd(cmd)
            if _looks_valid_file(output_path):
                if failures:
                    print(f"  OK → {os.path.basename(output_path)} ({tool_name}, with fallback)")
                else:
                    print(f"  OK → {os.path.basename(output_path)} ({tool_name})")
                return
            failures.append((tool_name, "small file", None))
        except subprocess.CalledProcessError as e:
            err = (e.stderr or "").strip()
            short_err = err.splitlines()[-1] if err else "no stderr"
            failures.append((tool_name, f"code {e.returncode}", short_err))

    print(f"  ERROR → {os.path.basename(output_path)} (all downloaders failed)")
    for tool_name, code_text, short_err in failures:
        msg = f"    - {tool_name}: {code_text}"
        if short_err:
            msg += f" | {short_err}"
        print(msg)
    raise RuntimeError(f"Download failed for {output_path}")


aria2c not found → installing...
aria2c installed successfully
Working directory: /kaggle/working


In [2]:
### CONTROLNET INSTALL OPTIONS ###

# Option A (default): force clean reinstall
# Option B: set CONTROLNET_INSTALL_MODE = "update" for git pull in existing repo
# Option C: set CONTROLNET_INSTALL_MODE = "skip" to keep current state

CONTROLNET_REPO_URL = "https://github.com/Mikubill/sd-webui-controlnet"
CONTROLNET_INSTALL_MODE = os.environ.get("CONTROLNET_INSTALL_MODE", "reinstall").strip().lower()

if CONTROLNET_INSTALL_MODE not in {"reinstall", "update", "skip"}:
    raise ValueError("CONTROLNET_INSTALL_MODE must be one of: reinstall, update, skip")

print(f"ControlNet extension path: {CONTROLNET_DIR}")
print(f"Install mode: {CONTROLNET_INSTALL_MODE}")

if CONTROLNET_INSTALL_MODE == "skip":
    print("ControlNet install skipped")
else:
    if os.path.isdir(CONTROLNET_DIR) and CONTROLNET_INSTALL_MODE == "reinstall":
        print("Removing existing ControlNet directory...")
        shutil.rmtree(CONTROLNET_DIR)

    if not os.path.isdir(CONTROLNET_DIR):
        print("Cloning ControlNet repository...")
        subprocess.run(["git", "clone", CONTROLNET_REPO_URL, CONTROLNET_DIR], check=True)
    else:
        print("Updating ControlNet repository...")
        subprocess.run(["git", "-C", CONTROLNET_DIR, "pull", "--ff-only"], check=True)

os.makedirs(CONTROLNET_MODELS_DIR, exist_ok=True)
print("ControlNet repository is ready")
print(f"ControlNet models directory: {CONTROLNET_MODELS_DIR}")

ControlNet extension path: /kaggle/working/stable-diffusion-webui-forge/extensions/sd-webui-controlnet
Install mode: reinstall
Removing existing ControlNet directory...
Cloning ControlNet repository...


Cloning into '/kaggle/working/stable-diffusion-webui-forge/extensions/sd-webui-controlnet'...


ControlNet repository is ready
ControlNet models directory: /kaggle/working/stable-diffusion-webui-forge/extensions/sd-webui-controlnet/models


In [3]:

#ControlNET models download

controlnet_models_to_download = [
    ("t2i-adapter_xl_openpose 151 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_openpose.safetensors", "t2i-adapter_xl_openpose.safetensors", "HF_TOKEN"),
    ("t2i-adapter_xl_canny 148 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_canny.safetensors", "t2i-adapter_xl_canny.safetensors", "HF_TOKEN"),
    ("t2i-adapter_xl_sketch 148 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_sketch.safetensors", "t2i-adapter_xl_sketch.safetensors", "HF_TOKEN"),
    ("t2i-adapter_diffusers_xl_depth_midas 151 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_depth_midas.safetensors", "t2i-adapter_diffusers_xl_depth_midas.safetensors", "HF_TOKEN"),
    ("t2i-adapter_diffusers_xl_depth_zoe 151 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_depth_zoe.safetensors", "t2i-adapter_diffusers_xl_depth_zoe.safetensors", "HF_TOKEN"),
    ("t2i-adapter_diffusers_xl_lineart 151 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_lineart.safetensors", "t2i-adapter_diffusers_xl_lineart.safetensors", "HF_TOKEN"),
    #("", "", "", ""),
]

print(f"Total Models/checkpoints in the list: {len(controlnet_models_to_download)}")
skipped = 0

print(f"All Models/checkpoints: {len(controlnet_models_to_download)}")
skipped = 0

for name, url, filename, source in controlnet_models_to_download:
    output_path = os.path.join(CONTROLNET_MODELS_DIR, filename)
    
    if os.path.exists(output_path) and os.path.getsize(output_path) > 1_000_000:
        print(f"  Already has → {name:<30} {filename}")
        skipped += 1
        continue
    
    token = TOKENS.get(source)
    
    print(f"  Loading {name:<30} ({source}) → {filename}")
    try:
        download_file(url=url, output_path=output_path, token=token)
    except Exception as e:
        print(f"  ERROR: {e}")

print(f"\n DONE. New: {len(controlnet_models_to_download) - skipped}, skipped: {skipped}")

Total Models/checkpoints in the list: 6
All Models/checkpoints: 6
  Loading t2i-adapter_xl_openpose 151 MB (HF_TOKEN) → t2i-adapter_xl_openpose.safetensors
  OK → t2i-adapter_xl_openpose.safetensors (aria2c)
  Loading t2i-adapter_xl_canny 148 MB    (HF_TOKEN) → t2i-adapter_xl_canny.safetensors
  OK → t2i-adapter_xl_canny.safetensors (aria2c)
  Loading t2i-adapter_xl_sketch 148 MB   (HF_TOKEN) → t2i-adapter_xl_sketch.safetensors
  OK → t2i-adapter_xl_sketch.safetensors (aria2c)
  Loading t2i-adapter_diffusers_xl_depth_midas 151 MB (HF_TOKEN) → t2i-adapter_diffusers_xl_depth_midas.safetensors
  OK → t2i-adapter_diffusers_xl_depth_midas.safetensors (aria2c)
  Loading t2i-adapter_diffusers_xl_depth_zoe 151 MB (HF_TOKEN) → t2i-adapter_diffusers_xl_depth_zoe.safetensors
  OK → t2i-adapter_diffusers_xl_depth_zoe.safetensors (aria2c)
  Loading t2i-adapter_diffusers_xl_lineart 151 MB (HF_TOKEN) → t2i-adapter_diffusers_xl_lineart.safetensors
  OK → t2i-adapter_diffusers_xl_lineart.safetensors (a

In [4]:

#controlnet models to download

models_to_download = [
    ("WAI ILL V16.0 6,46 GB", "https://civitai.com/api/download/models/2514310?type=Model&format=SafeTensor&size=pruned&fp=fp16", "waiIllustriousSDXL_v160.safetensors", "CIVITAI"),
]

print(f"Total Models/checkpoints in the list: {len(models_to_download)}")
skipped = 0

print(f"All Models/checkpoints: {len(models_to_download)}")
skipped = 0

for name, url, filename, source in models_to_download:
    output_path = os.path.join(MODELS_DIR, filename)
    
    if os.path.exists(output_path) and os.path.getsize(output_path) > 1_000_000:
        print(f"  Already has → {name:<30} {filename}")
        skipped += 1
        continue
    
    token = TOKENS.get(source)
    
    print(f"  Loading {name:<30} ({source}) → {filename}")
    try:
        download_file(url=url, output_path=output_path, token=token)
    except Exception as e:
        print(f"  ERROR: {e}")

print(f"\n DONE. New: {len(models_to_download) - skipped}, skipped: {skipped}")

Total Models/checkpoints in the list: 1
All Models/checkpoints: 1
  Loading WAI ILL V16.0 6,46 GB          (CIVITAI) → waiIllustriousSDXL_v160.safetensors
  OK → waiIllustriousSDXL_v160.safetensors (curl)

 DONE. New: 1, skipped: 0


In [5]:

#LoRa

lora_to_download = [
    ("Detailer IL V2 218 MB",        "https://civitai.com/api/download/models/1736373?type=Model&format=SafeTensor",    "detailer_v2_il.safetensors",     "CIVITAI"),
    ("Realistic filter V1 55 MB",    "https://civitai.com/api/download/models/1124771?type=Model&format=SafeTensor",    "realistic_filter_v1_il.safetensors", "CIVITAI"),
    ("Hyperrealistic V4 ILL 435 MB", "https://civitai.com/api/download/models/1914557?type=Model&format=SafeTensor",    "hyperrealistic_v4_ill.safetensors",  "CIVITAI"),
    ("Niji semi realism V3.5 ILL 435 MB", "https://civitai.com/api/download/models/1882710?type=Model&format=SafeTensor", "niji_semi_realism_v35.safetensors", "CIVITAI"), 
    ("ATNR Style ILL V1.1 350 MB", "https://civitai.com/api/download/models/1711464?type=Model&format=SafeTensor", "atnr_style_ill_v1.1.safetensors", "CIVITAI"), 
    ("Face Enhancer Ill 218 MB", "https://civitai.com/api/download/models/1839268?type=Model&format=SafeTensor", "face_enhancer_ill.safetensors", "CIVITAI"),
    ("Smooth Detailer Booster V4 243 MB", "https://civitai.com/api/download/models/2196453?type=Model&format=SafeTensor", "smooth_detailer_booster_v4.safetensors", "CIVITAI"),
    ("USNR Style V-pred 157 MB", "https://civitai.com/api/download/models/2555444?type=Model&format=SafeTensor", "usnr_style_v_pred.safetensors", "CIVITAI"),
    ("748cm Style V1 243 MB", "https://civitai.com/api/download/models/1056404?type=Model&format=SafeTensor", "748cm_style_v1.safetensors", "CIVITAI"),
    ("Velvet's Mythic Fantasy Styles IL 218 MB", "https://civitai.com/api/download/models/2620790?type=Model&format=SafeTensor", "velvets_mythic_fantasy_styles_il.safetensors", "CIVITAI"),
    #("", "", "", "CIVITAI"),
]

print(f"Total LoRa in the list: {len(lora_to_download)}")
skipped = 0

print(f"All LoRA: {len(lora_to_download)}")
skipped = 0

for name, url, filename, source in lora_to_download:
    output_path = os.path.join(LORA_DIR, filename)
    
    if os.path.exists(output_path) and os.path.getsize(output_path) > 1_000_000:
        print(f"  Already has → {name:<30} {filename}")
        skipped += 1
        continue
    
    token = TOKENS.get(source)
    
    print(f"  Loading {name:<30} ({source}) → {filename}")
    try:
        download_file(url=url, output_path=output_path, token=token)
    except Exception as e:
        print(f"  ERROR: {e}")

print(f"\n DONE. New: {len(lora_to_download) - skipped}, skipped: {skipped}")

Total LoRa in the list: 10
All LoRA: 10
  Loading Detailer IL V2 218 MB          (CIVITAI) → detailer_v2_il.safetensors
  OK → detailer_v2_il.safetensors (curl)
  Loading Realistic filter V1 55 MB      (CIVITAI) → realistic_filter_v1_il.safetensors
  OK → realistic_filter_v1_il.safetensors (curl)
  Loading Hyperrealistic V4 ILL 435 MB   (CIVITAI) → hyperrealistic_v4_ill.safetensors
  OK → hyperrealistic_v4_ill.safetensors (curl)
  Loading Niji semi realism V3.5 ILL 435 MB (CIVITAI) → niji_semi_realism_v35.safetensors
  OK → niji_semi_realism_v35.safetensors (curl)
  Loading ATNR Style ILL V1.1 350 MB     (CIVITAI) → atnr_style_ill_v1.1.safetensors
  OK → atnr_style_ill_v1.1.safetensors (curl)
  Loading Face Enhancer Ill 218 MB       (CIVITAI) → face_enhancer_ill.safetensors
  OK → face_enhancer_ill.safetensors (curl)
  Loading Smooth Detailer Booster V4 243 MB (CIVITAI) → smooth_detailer_booster_v4.safetensors
  OK → smooth_detailer_booster_v4.safetensors (curl)
  Loading USNR Style V-